# Preparam el notebook

In [5]:
import os

import pandas as pd

from skimage import io
from skimage.transform import resize

from sklearn.feature_extraction import image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Llegim totes les imatges del dataset

In [6]:
def get_image_files_by_folder(root_folder, image_extensions=('jpg', 'jpeg', 'png', 'gif', 'bmp')):
    image_files_by_folder = {}

    for folder_path, _, files in os.walk(root_folder):
        folder_name = os.path.basename(folder_path).lower()
        image_files = []

        for file in files:
            file_extension = file.lower().split('.')[-1]
            if file_extension in image_extensions:
                image_files.append(os.path.join(folder_path, file))

        if image_files:
            image_files_by_folder[folder_name] = image_files

    return image_files_by_folder

In [7]:
train_dataset_path = "data/train"
images_to_load = get_image_files_by_folder(train_dataset_path)
images_to_load.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

# Cream el dataframe

In [10]:
def read_and_flatten_images(image_list):
    image_data = []
    
    for image_path in image_list:
        img = io.imread(image_path, as_gray=True)
        img = resize(img, (200, 200))
        image_data.append(img.flatten())
    
    return image_data

In [11]:
image_df = pd.DataFrame()
for label, image_list in images_to_load.items():
    image_data = read_and_flatten_images(images_to_load[label])
    next_part = pd.DataFrame(image_data)
    next_part['label'] = label
    image_df = pd.concat([image_df, next_part], ignore_index=True)

In [12]:
image_df.shape

(1500, 40001)

# Cream els conjunts de test i d'entrenament

In [13]:
X_train, X_test, y_train, y_test = train_test_split(image_df.drop('label', axis=1), image_df['label'], test_size=0.20, random_state=42)

# Entrenam el model

In [14]:
svm = SVC(C=1.0, kernel='linear', random_state=42)
svm.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

# Donam una predicció

In [15]:
y_predicted = svm.predict(X_test)

print(f"La precisició es de: {accuracy_score(y_test, y_predicted)}")

La precisició es de: 0.23333333333333334
